In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")

In [ ]:
# 데이터 로드 및 기본 정보 확인
print("=" * 50)
print("XGBoost 건설업 재무지표 예측 모델")
print("=" * 50)

# 데이터 로드
data = pd.read_csv('../preprocessing/integrated_data.csv')
data['date'] = pd.to_datetime(data['date'])

print(f"데이터 크기: {data.shape}")
print(f"기간: {data['date'].min()} ~ {data['date'].max()}")
print(f"기업 수: {data['corp_name'].nunique()}")
print(f"컬럼 수: {len(data.columns)}")

# 데이터 구조 확인
print("\n컬럼 목록:")
for i, col in enumerate(data.columns):
    print(f"{i+1:2d}. {col}")

data.head()

In [ ]:
# 타겟 변수 및 피처 정의
target_columns = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '분기순이익']

# 경제지표 피처 (ECOS 데이터)
economic_features = [
    'base_rate', 'ccsi', 'construction_bsi_actual', 'cpi', 'esi',
    'exchange_usd_원_달러종가_15_30', 'housing_lease_price', 'housing_sale_price',
    'import_price_비금속광물', 'import_price_철강1차제품', 'leading_index',
    'm2_growth', 'market_rate_국고채10년', 'market_rate_국고채3년',
    'market_rate_회사채3년_AA_', 'market_rate_회사채3년_BBB_',
    'ppi_비금속광물', 'ppi_철강1차제품'
]

# 파생 재무지표 피처
financial_features = [
    '부채비율', '자기자본비율', 'ROA', 'ROE',
    '매출액성장률', '영업이익성장률', '순이익성장률'
]

# 전체 피처
feature_columns = economic_features + financial_features

print(f"타겟 변수: {len(target_columns)}개")
print(f"경제지표 피처: {len(economic_features)}개")
print(f"재무지표 피처: {len(financial_features)}개")
print(f"전체 피처: {len(feature_columns)}개")

# 데이터 품질 확인
print("\n결측치 확인:")
missing_data = data[target_columns + feature_columns].isnull().sum()
print(missing_data[missing_data > 0])

In [ ]:
# 데이터 전처리 및 시계열 분할
def prepare_data(data, feature_columns, target_columns):
    """데이터 전처리 및 시계열 분할"""
    
    # 결측치 처리
    data_clean = data.copy()
    data_clean = data_clean.dropna(subset=target_columns + feature_columns)
    
    # 시간순 정렬
    data_clean = data_clean.sort_values(['corp_name', 'date']).reset_index(drop=True)
    
    # 80% 훈련, 20% 테스트 (시계열 순서 유지)
    unique_dates = sorted(data_clean['date'].unique())
    split_idx = int(len(unique_dates) * 0.8)
    train_dates = unique_dates[:split_idx]
    test_dates = unique_dates[split_idx:]
    
    train_data = data_clean[data_clean['date'].isin(train_dates)]
    test_data = data_clean[data_clean['date'].isin(test_dates)]
    
    print(f"전체 데이터: {len(data_clean)}개")
    print(f"훈련 데이터: {len(train_data)}개 ({train_dates[0].strftime('%Y-%m')} ~ {train_dates[-1].strftime('%Y-%m')})")
    print(f"테스트 데이터: {len(test_data)}개 ({test_dates[0].strftime('%Y-%m')} ~ {test_dates[-1].strftime('%Y-%m')})")
    
    return train_data, test_data, train_dates, test_dates

# 데이터 준비
train_data, test_data, train_dates, test_dates = prepare_data(data, feature_columns, target_columns)

# 피처 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[feature_columns])
X_test = scaler.transform(test_data[feature_columns])

print(f"\n피처 스케일링 완료")
print(f"훈련 피처 형태: {X_train.shape}")
print(f"테스트 피처 형태: {X_test.shape}")

In [ ]:
# XGBoost 모델 훈련
def train_xgboost_models(X_train, train_data, target_columns):
    """각 타겟별 XGBoost 모델 훈련"""
    
    models = {}
    training_scores = {}
    
    # XGBoost 하이퍼파라미터
    xgb_params = {
        'n_estimators': 200,
        'max_depth': 6,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'n_jobs': -1
    }
    
    print("XGBoost 모델 훈련 시작")
    print("-" * 40)
    
    for target in target_columns:
        print(f"훈련 중: {target}")
        
        # 타겟 데이터 준비
        y_train = train_data[target].values
        
        # 모델 훈련
        model = xgb.XGBRegressor(**xgb_params)
        model.fit(X_train, y_train)
        
        # 훈련 성능 평가
        y_train_pred = model.predict(X_train)
        train_r2 = r2_score(y_train, y_train_pred)
        train_mae = mean_absolute_error(y_train, y_train_pred)
        
        models[target] = model
        training_scores[target] = {
            'R2': train_r2,
            'MAE': train_mae
        }
        
        print(f"  훈련 R2: {train_r2:.4f}, MAE: {train_mae:,.0f}")
    
    return models, training_scores

# 모델 훈련 실행
models, training_scores = train_xgboost_models(X_train, train_data, target_columns)

In [ ]:
# 모델 평가
def evaluate_models(models, X_test, test_data, target_columns):
    """테스트 데이터로 모델 성능 평가"""
    
    test_scores = {}
    predictions = {}
    
    print("모델 성능 평가")
    print("=" * 60)
    print(f"{'지표':<12} {'훈련 R2':<10} {'테스트 R2':<11} {'테스트 MAE':<15}")
    print("-" * 60)
    
    for target in target_columns:
        # 예측
        y_test = test_data[target].values
        y_pred = models[target].predict(X_test)
        
        # 성능 지표 계산
        test_r2 = r2_score(y_test, y_pred)
        test_mae = mean_absolute_error(y_test, y_pred)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        test_scores[target] = {
            'R2': test_r2,
            'MAE': test_mae,
            'RMSE': test_rmse
        }
        
        predictions[target] = y_pred
        
        # 결과 출력
        train_r2 = training_scores[target]['R2']
        print(f"{target:<12} {train_r2:<10.4f} {test_r2:<11.4f} {test_mae:<15,.0f}")
    
    return test_scores, predictions

# 평가 실행
test_scores, predictions = evaluate_models(models, X_test, test_data, target_columns)

In [ ]:
# 피처 중요도 분석
def analyze_feature_importance(models, feature_columns, target_columns):
    """피처 중요도 분석 및 시각화"""
    
    plt.figure(figsize=(15, 10))
    
    for i, target in enumerate(target_columns):
        plt.subplot(2, 3, i+1)
        
        # 피처 중요도 추출
        importance = models[target].feature_importances_
        feature_importance = pd.DataFrame({
            'feature': feature_columns,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
        # 상위 10개 피처만 시각화
        top_features = feature_importance.head(10)
        
        plt.barh(range(len(top_features)), top_features['importance'])
        plt.yticks(range(len(top_features)), top_features['feature'])
        plt.xlabel('중요도')
        plt.title(f'{target} 피처 중요도')
        plt.gca().invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    return feature_importance

# 피처 중요도 분석 실행
analyze_feature_importance(models, feature_columns, target_columns)

In [ ]:
# 다음 분기 재무지표 예측
def predict_next_quarter(models, data, scaler, feature_columns, target_columns):
    """월별 데이터를 사용하여 다음 분기 재무지표 예측"""
    
    print("다음 분기 재무지표 예측")
    print("=" * 40)
    
    # 데이터셋의 마지막 날짜 확인
    last_date = data['date'].max()
    print(f"데이터셋 마지막 날짜: {last_date.strftime('%Y-%m-%d')}")
    
    # 다음 분기 계산
    last_year = last_date.year
    last_month = last_date.month
    
    # 분기 계산 (1-3월=1Q, 4-6월=2Q, 7-9월=3Q, 10-12월=4Q)
    last_quarter = (last_month - 1) // 3 + 1
    
    # 다음 분기 계산
    if last_quarter == 4:
        next_year = last_year + 1
        next_quarter = 1
    else:
        next_year = last_year
        next_quarter = last_quarter + 1
    
    next_quarter_str = f"{next_year}년 {next_quarter}분기"
    print(f"예측 대상: {next_quarter_str}")
    
    # 최신 월별 데이터 사용 (각 기업별로)
    predictions = []
    companies = data['corp_name'].unique()
    
    for corp in companies:
        corp_data = data[data['corp_name'] == corp].sort_values('date')
        
        if len(corp_data) == 0:
            continue
            
        # 최신 데이터 사용
        latest_data = corp_data.iloc[-1]
        
        # 피처 준비
        features = latest_data[feature_columns].values.reshape(1, -1)
        features_scaled = scaler.transform(features)
        
        # 각 타겟별 예측
        corp_pred = {'corp_name': corp}
        
        for target in target_columns:
            pred_value = models[target].predict(features_scaled)[0]
            corp_pred[f'{target}_예측'] = pred_value
        
        predictions.append(corp_pred)
    
    # 결과 데이터프레임 생성
    pred_df = pd.DataFrame(predictions)
    
    # 결과 출력
    print(f"\n{next_quarter_str} 재무지표 예측 결과:")
    print("-" * 40)
    
    for target in target_columns:
        col_name = f'{target}_예측'
        if col_name in pred_df.columns:
            mean_pred = pred_df[col_name].mean()
            print(f"{target}: 평균 {mean_pred:,.0f}")
    
    return pred_df

# 다음 분기 예측 실행
next_quarter_pred = predict_next_quarter(models, data, scaler, feature_columns, target_columns)
next_quarter_pred

In [ ]:
# 결과 저장 및 요약
def save_results(next_quarter_pred, test_scores, training_scores):
    """예측 결과 및 성능 지표 저장"""
    
    import os
    os.makedirs('../results', exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. 예측 결과 저장
    pred_file = f'../results/quarterly_predictions_{timestamp}.csv'
    next_quarter_pred.to_csv(pred_file, index=False, encoding='utf-8-sig')
    
    # 2. 성능 지표 저장
    performance_data = []
    for target in target_columns:
        perf_row = {
            '지표': target,
            '훈련_R2': training_scores[target]['R2'],
            '테스트_R2': test_scores[target]['R2'],
            '테스트_MAE': test_scores[target]['MAE'],
            '테스트_RMSE': test_scores[target]['RMSE']
        }
        performance_data.append(perf_row)
    
    perf_df = pd.DataFrame(performance_data)
    perf_file = f'../results/model_performance_{timestamp}.csv'
    perf_df.to_csv(perf_file, index=False, encoding='utf-8-sig')
    
    # 3. 요약 보고서
    print("XGBoost 모델 성능 요약")
    print("=" * 50)
    print(f"전체 평균 테스트 R2: {perf_df['테스트_R2'].mean():.4f}")
    print(f"최고 성능 지표: {perf_df.loc[perf_df['테스트_R2'].idxmax(), '지표']} (R2: {perf_df['테스트_R2'].max():.4f})")
    print(f"개선 필요 지표: {perf_df.loc[perf_df['테스트_R2'].idxmin(), '지표']} (R2: {perf_df['테스트_R2'].min():.4f})")
    
    print(f"\n결과 파일 저장:")
    print(f"예측 결과: {pred_file}")
    print(f"성능 지표: {perf_file}")
    
    return perf_df

# 결과 저장
performance_summary = save_results(next_quarter_pred, test_scores, training_scores)
performance_summary